# 🎯 GitHub मॉडेल्स (.NET) सह नियोजन आणि डिझाइन पॅटर्न

## 📋 शिकण्याची उद्दिष्टे

या नोटबुकमध्ये Microsoft Agent Framework चा वापर करून GitHub मॉडेल्ससह बुद्धिमान एजंट तयार करण्यासाठी एंटरप्राइझ-ग्रेड नियोजन आणि डिझाइन पॅटर्न प्रदर्शित केले आहेत. तुम्ही अशा एजंट्स तयार करायला शिकाल जे जटिल समस्या विभागू शकतात, बहु-चरणीय उपाययोजना आखू शकतात आणि .NET च्या एंटरप्राइझ वैशिष्ट्यांसह प्रगत कार्यप्रवाह अंमलात आणू शकतात.

## ⚙️ पूर्वतयारी आणि सेटअप

**विकसन वातावरण:**
- .NET 9.0 SDK किंवा त्याहून अधिक
- Visual Studio 2022 किंवा VS Code (C# विस्तारासह)
- GitHub मॉडेल्स API प्रवेश

**आवश्यक अवलंबित्व:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**पर्यावरण कॉन्फिगरेशन (.env फाइल):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// #r "nuget: OpenAI, 2.4.0"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0/Microsoft.Agents.AI.OpenAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [5]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;


In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
System.Environment.GetEnvironmentVariable("GITHUB_ENDPOINT");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:


var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string AGENT_NAME ="TravelPlanAgent";

const string AGENT_INSTRUCTIONS = @"You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general request";

In [15]:

    public class Plan
    {
        [JsonPropertyName("assigned_agent")]
        public string? Assigned_agent { get; set; }

        [JsonPropertyName("task_details")]
        public string? Task_details { get; set; }
    }

In [16]:

    public class TravelPlan
    {
        [JsonPropertyName("main_task")]
        public string? Main_task { get; set; }

        [JsonPropertyName("subtasks")]
        public IList<Plan> Subtasks { get; set; }

    }

In [17]:

ChatClientAgentOptions agentOptions = new(name: AGENT_NAME, instructions: AGENT_INSTRUCTIONS)
{
    ChatOptions = new()
    {
        ResponseFormat = ChatResponseFormatJson.ForJsonSchema(
            schema: AIJsonUtilities.CreateJsonSchema(typeof(TravelPlan)),
            schemaName: "TravelPlan",
            schemaDescription: "Travel Plan with main_task and subtasks")
    }
};

In [18]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    agentOptions);

In [19]:

Console.WriteLine(await agent.RunAsync("Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne"));

{"main_task":"Plan a family trip from Singapore to Melbourne","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Find and book flights from Singapore to Melbourne for a family of 4."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne."},{"assigned_agent":"CarRental","task_details":"Arrange car rental in Melbourne for local transportation."},{"assigned_agent":"ActivitiesBooking","task_details":"Book family-friendly activities and attractions in Melbourne."},{"assigned_agent":"DestinationInfo","task_details":"Provide information about Melbourne, including dining and family-friendly areas."}]}



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने ग्रस्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
